In [14]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd

def get_db_connection():
    server = SSHTunnelForwarder(
        ('letsrise.myonline.works', 22),  # SSH server and port
        ssh_username='ubuntu',  # SSH username
        ssh_pkey='~/.ssh/id_letsrise',  # Path to your private key
        remote_bind_address=('localhost', 5432)  # Database server and port
    )

    server.start()

    conn = psycopg2.connect(
        dbname='letsrise_v1',
        user='letsrise_intern',
        password='letsrise',
        host='localhost',
        port=server.local_bind_port  # Use the dynamically assigned local port
    )
    return conn, server

In [19]:
conn, server = get_db_connection()
query1 = """
SELECT * FROM public.user_info
ORDER BY user_id ASC 
"""
df = pd.read_sql(query1, conn)
conn.close()
server.stop()

In [20]:
df.columns

Index(['user_id', 'name', 'email', 'age', 'linkedin_url', 'education_level',
       'employment_status', 'entrepreneurial_experience',
       'current_startup_stage', 'number_of_startups',
       'role_in_entrepreneurship', 'industry_experience',
       'number_of_previous_startups', 'location', 'gender', 'startup_name'],
      dtype='object')

In [22]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the Dash app with the given color palette
app.layout = html.Div(style={'backgroundColor': '#f7f7f7', 'fontFamily': 'Poppins, sans-serif'}, children=[
    dcc.Dropdown(
        id='user-dropdown',
        options=[{'label': name, 'value': user_id} for user_id, name in zip(df['user_id'], df['name'])],
        placeholder="Select a User",
        style={'marginBottom': '10px'}
    ),
    dcc.Dropdown(
        id='type-dropdown',
        options=[
            {'label': 'Business Partner', 'value': 'business_partner'},
            {'label': 'Mentor', 'value': 'mentor'}
        ],
        placeholder="Select Type",
        style={'marginBottom': '10px'}
    ),
    dcc.Dropdown(
        id='industry-dropdown',
        placeholder="Select Industry Experience",
        style={'marginBottom': '20px'}
    ),
    html.Div(id='results', style={'color': '#111', 'padding': '20px', 'border': '1px solid #ccc', 'borderRadius': '5px', 'boxShadow': '0 1px 3px rgba(0, 0, 0, 0.1)', 'backgroundColor': '#fff'}),
])

# Callback to update the industry dropdown based on the unique values in the DataFrame
@app.callback(
    Output('industry-dropdown', 'options'),
    Input('type-dropdown', 'value')
)
def set_industry_options(selected_type):
    if selected_type:
        unique_industries = df['industry_experience'].dropna().unique()
        return [{'label': industry, 'value': industry} for industry in unique_industries]
    return []

# Callback to update the results based on selected user, type, and industry experience
@app.callback(
    Output('results', 'children'),
    [Input('user-dropdown', 'value'),
     Input('type-dropdown', 'value'),
     Input('industry-dropdown', 'value')]
)
def update_results(selected_user, selected_type, selected_industry):
    if not selected_user or not selected_type or not selected_industry:
        return "Please make all selections."

    filtered_df = df[df['industry_experience'] == selected_industry]
    
    if selected_type == 'business_partner':
        top_5 = filtered_df[filtered_df['user_id'] != selected_user].head(5)
    elif selected_type == 'mentor':
        top_5 = filtered_df[filtered_df['user_id'] != selected_user].sort_values(by='entrepreneurial_experience', ascending=False).head(5)
    
    return html.Ul([
        html.Li([
            f"{row['name']} - {row['industry_experience']} - {row['entrepreneurial_experience']} years of experience"
        ]) for idx, row in top_5.iterrows()
    ])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [26]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from email.mime.text import MIMEText
import base64

# Path to your service account key file for Gmail API
key_file_path = 'letsrise-caed029c5496.json'

# Create credentials
credentials = service_account.Credentials.from_service_account_file(
    key_file_path, scopes=['https://www.googleapis.com/auth/gmail.send'])

try:
    # Build the Gmail service
    service = build('gmail', 'v1', credentials=credentials)
    print("Successfully authenticated with Gmail API.")

    # Example: Sending an email
    to_email = 'svn9705@nyu.edu'
    subject = 'Test Email'
    message_text = 'This is a test email sent from Gmail API.'

    message = MIMEText(message_text)
    message['to'] = to_email
    message['from'] = 'svn9705@nyu.edu'
    message['subject'] = subject
    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

    # Send the email
    sent_message = service.users().messages().send(userId='me', body={'raw': raw_message}).execute()
    print('Message Id: %s' % sent_message['id'])

except Exception as e:
    print(f"Error occurred: {e}")


Successfully authenticated with Gmail API.
Error occurred: <HttpError 400 when requesting https://gmail.googleapis.com/gmail/v1/users/me/messages/send?alt=json returned "Precondition check failed.". Details: "[{'message': 'Precondition check failed.', 'domain': 'global', 'reason': 'failedPrecondition'}]">
